## Preprocessing

In [97]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.metrics import accuracy_score
import xgboost as xgb

import pandas as pd
import tensorflow as tf
pd.set_option('display.max_rows', None)

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [49]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df = application_df.drop(columns=['EIN','NAME', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT'])
application_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,1


In [50]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
application_df.nunique()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
STATUS               2
INCOME_AMT           9
IS_SUCCESSFUL        2
dtype: int64

In [51]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [52]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T13','T12','T2','T25','T14','T29','T15','T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [53]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
class_values = application_df['CLASSIFICATION'].value_counts()

In [54]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = class_values[class_values < 1883].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [56]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns = [
    'APPLICATION_TYPE', 
    'AFFILIATION', 
    'CLASSIFICATION', 
    'USE_CASE', 
    'ORGANIZATION', 
    'INCOME_AMT', 
]

application_df_encoded = pd.get_dummies(application_df, columns=categorical_columns,drop_first=True)
application_df_encoded.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,1,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,1,False,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,1,0,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1,1,False,False,True,False,False,False,False,False,...,False,True,False,True,False,False,False,False,False,False
4,1,1,False,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,False,False,False


In [57]:
# Split our preprocessed data into our features and target arrays
y = application_df_encoded['IS_SUCCESSFUL']

X = application_df_encoded.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)



In [58]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [83]:
# Define the model - deep neural net
nn = tf.keras.models.Sequential()

# Input Layer 
nn.add(tf.keras.layers.Dense(units=64, activation='relu', input_dim=X_train_scaled.shape[1]))

# First hidden layer 
nn.add(tf.keras.layers.Dense(units=32, activation='relu'))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation='relu'))

# Output layer with sigmoid activation (binary classification)
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_49 (Dense)                │ (None, 64)             │         2,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,929 (19.25 KB)

 Trainable params: 4,929 (19.25 KB)

 Non-trainable params: 0 (0.00 B)

In [100]:
# Compile the model
optimizer = Adam(learning_rate=0.001)
nn.compile(optimizer=optimizer,
           loss='binary_crossentropy',
           metrics=['accuracy'])

In [106]:

# Train the model
fit_model = nn.fit(
    X_train_scaled, y_train,
    epochs=100, # Number of iterations
    batch_size=32, # Number of samples per gradient update
)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 900us/step - accuracy: 0.7328 - loss: 0.5389
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 818us/step - accuracy: 0.7400 - loss: 0.5377
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 857us/step - accuracy: 0.7352 - loss: 0.5387
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 783us/step - accuracy: 0.7411 - loss: 0.5340
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 816us/step - accuracy: 0.7405 - loss: 0.5321
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 922us/step - accuracy: 0.7383 - loss: 0.5342
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 828us/step - accuracy: 0.7376 - loss: 0.5365
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 827us/step - accuracy: 0.7413 - loss: 0.5308
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 848us/step - accuracy: 0.7422 - loss: 0.5305
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 829us/step - accuracy: 0.7380 - loss: 0.5358
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 822us/step - accuracy: 0.7431 - loss: 0.5337
Epoch 12/100
804/80

In [107]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 521us/step - accuracy: 0.7285 - loss: 0.6091
Loss: 0.6090608835220337, Accuracy: 0.7285131216049194


In [108]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')

# OPTIONAL:Save model using native Keras format
nn.save('AlphabetSoupCharity.keras')
